In [1]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import matplotlib.pyplot as plt
import math as mt
fp_padrao = 0.92
fp_padrao_Sup = 2-fp_padrao

In [2]:
def normaliza(s):
    if s >= 0:
       return s
    elif s < 0 :
       return s + 2

In [3]:
universal01_dataset = pd.read_excel('universal.xlsx', 'Universal_01')
universal02_dataset = pd.read_excel('universal.xlsx', 'Universal_02')
univ01 = universal01_dataset
univ02 = universal02_dataset

Universal = pd.concat([univ02,univ01])

Universal.drop(columns=['Freqüência', 'Ocorrência'],inplace=True)

Universal.rename(columns ={'Fat,Pot T': 'FP T', 
                            'Fat,Pot, R': 'FP R',
                            'Fat,Pot, S': 'FP S'}, inplace=True)
Universal_p = pd.pivot_table(Universal, index=['Data'])

In [4]:
Universal_p['Pot R KVA'] = (Universal_p['Corrente R'] * Universal_p['Tensão R'] * (mt.sqrt(3)))/1000
Universal_p['Pot S KVA'] = (Universal_p['Corrente S'] * Universal_p['Tensão S'] * (mt.sqrt(3)))/1000
Universal_p['Pot T KVA'] = (Universal_p['Corrente T'] * Universal_p['Tensão T'] * (mt.sqrt(3)))/1000

Universal['Pot R KVA'] = (Universal['Corrente R'] * Universal['Tensão R'] * (mt.sqrt(3)))/1000
Universal['Pot S KVA'] = (Universal['Corrente S'] * Universal['Tensão S'] * (mt.sqrt(3)))/1000
Universal['Pot T KVA'] = (Universal['Corrente T'] * Universal['Tensão T'] * (mt.sqrt(3)))/1000

In [5]:
#Calculo KVAr usando numpy
Universal_p['Pot R KW'] = (Universal_p['Pot R KVA'] * (Universal_p['FP R']))
Universal_p['Pot S KW'] = (Universal_p['Pot S KVA'] * (Universal_p['FP S']))
Universal_p['Pot T KW'] = (Universal_p['Pot T KVA'] * (Universal_p['FP T']))

Universal['Pot R KW'] = (Universal['Pot R KVA'] * (Universal['FP R']))
Universal['Pot S KW'] = (Universal['Pot S KVA'] * (Universal['FP S']))
Universal['Pot T KW'] = (Universal['Pot T KVA'] * (Universal['FP T']))

In [6]:
#Calculo KVAr usando numpy
Universal_p['Pot R KVAr'] = np.sqrt(np.power(Universal_p['Pot R KVA'],2) - np.power(Universal_p['Pot R KW'],2))
Universal_p['Pot S KVAr'] = np.sqrt(np.power(Universal_p['Pot S KVA'],2) - np.power(Universal_p['Pot S KW'],2))
Universal_p['Pot T KVAr'] = np.sqrt(np.power(Universal_p['Pot T KVA'],2) - np.power(Universal_p['Pot T KW'],2))

Universal['Pot R KVAr'] = np.sqrt(np.power(Universal['Pot R KVA'],2) - np.power(Universal['Pot R KW'],2))
Universal['Pot S KVAr'] = np.sqrt(np.power(Universal['Pot S KVA'],2) - np.power(Universal['Pot S KW'],2))
Universal['Pot T KVAr'] = np.sqrt(np.power(Universal['Pot T KVA'],2) - np.power(Universal['Pot T KW'],2))

In [7]:
#Calculo Faltante Kvar
Universal_p['Pot R KVAr faltante'] = (Universal_p['Pot R KVAr']) - (np.sqrt(np.power(Universal_p['Pot R KVA'],2)-
                                                                      (np.power((Universal_p['Pot R KVA'] * (fp_padrao)),2))))
Universal_p['Pot S KVAr faltante'] = (Universal_p['Pot S KVAr']) - (np.sqrt(np.power(Universal_p['Pot S KVA'],2)-
                                                                      (np.power((Universal_p['Pot S KVA'] * (fp_padrao)),2))))
Universal_p['Pot T KVAr faltante'] = (Universal_p['Pot T KVAr']) - (np.sqrt(np.power(Universal_p['Pot T KVA'],2)-
                                                                      (np.power((Universal_p['Pot T KVA'] * (fp_padrao)),2))))

Universal['Pot R KVAr faltante'] = (Universal['Pot R KVAr']) - (np.sqrt(np.power(Universal['Pot R KVA'],2)-
                                                                      (np.power((Universal['Pot R KVA'] * (fp_padrao)),2))))
Universal['Pot S KVAr faltante'] = (Universal['Pot S KVAr']) - (np.sqrt(np.power(Universal['Pot S KVA'],2)-
                                                                      (np.power((Universal['Pot S KVA'] * (fp_padrao)),2))))
Universal['Pot T KVAr faltante'] = (Universal['Pot T KVAr']) - (np.sqrt(np.power(Universal['Pot T KVA'],2)-
                                                                      (np.power((Universal['Pot T KVA'] * (fp_padrao)),2))))

In [8]:
Universal["FP R"] = Universal["FP R"].apply(normaliza)
Universal["FP S"] = Universal["FP S"].apply(normaliza)
Universal["FP T"] = Universal["FP T"].apply(normaliza)

Universal_p["FP R"] = Universal_p["FP R"].apply(normaliza)
Universal_p["FP S"] = Universal_p["FP S"].apply(normaliza)
Universal_p["FP T"] = Universal_p["FP T"].apply(normaliza)

In [9]:
fpfases_abaixo = Universal.loc[((Universal["FP R"]< fp_padrao) | (Universal["FP R"]> fp_padrao_Sup))
                         | ((Universal["FP S"]< fp_padrao) | (Universal["FP S"]> fp_padrao_Sup))
                         | ((Universal["FP T"]< fp_padrao) | (Universal["FP T"]> fp_padrao_Sup))]

fpfases = Universal_p.loc[((Universal_p["FP R"]< fp_padrao) | (Universal_p["FP R"]> fp_padrao_Sup))
                         | ((Universal_p["FP S"]< fp_padrao) | (Universal_p["FP S"]> fp_padrao_Sup))
                         | ((Universal_p["FP T"]< fp_padrao) | (Universal_p["FP T"]> fp_padrao_Sup))]

In [10]:
maximoR = fpfases['Pot R KVAr faltante'].max()
maximoS = fpfases['Pot S KVAr faltante'].max()
maximoT = fpfases['Pot T KVAr faltante'].max()

In [11]:
Universal.head()

,Data,Tensão R,Corrente R,FP R,Tensão S,Corrente S,FP S,Tensão T,Corrente T,FP T,...,Pot T KVA,Pot R KW,Pot S KW,Pot T KW,Pot R KVAr,Pot S KVAr,Pot T KVAr,Pot R KVAr faltante,Pot S KVAr faltante,Pot T KVAr faltante
0,2019-07-11 09:02:50,215.75,212.00,1.01,213.84,225.00,1.01,213.95,237.50,1.00,...,88.010914,-78.430049,-82.502534,88.010914,11.175677,11.755974,0.000000,-19.872986,-20.904892,-34.493093
1,2019-07-11 09:04:10,216.00,499.75,1.01,214.23,513.25,1.00,213.81,572.75,1.01,...,212.106383,-185.098277,190.445131,-209.985319,26.375076,0.000000,29.921287,-46.901099,-74.638943,-53.207098
2,2019-07-11 09:04:20,215.59,222.50,1.00,214.10,260.50,1.00,213.70,183.50,1.00,...,67.920554,83.084355,96.601756,67.920554,0.000000,0.000000,0.000000,-32.562284,-37.860002,-26.619312
3,2019-07-11 09:04:30,215.78,285.25,1.00,214.20,264.00,1.00,213.78,245.00,1.00,...,90.718066,106.609884,97.945395,90.718066,0.000000,0.000000,0.000000,-41.782371,-38.386598,-35.554076
4,2019-07-11 09:04:40,216.00,274.25,1.00,214.46,239.00,1.00,214.06,247.00,1.00,...,91.578411,102.603226,88.777892,91.578411,0.000000,0.000000,0.000000,-40.212088,-34.793686,-35.891260


In [12]:
Universal_p.head()

,Corrente R,Corrente S,Corrente T,FP R,FP S,FP T,Tensão R,Tensão S,Tensão T,Pot R KVA,...,Pot T KVA,Pot R KW,Pot S KW,Pot T KW,Pot R KVAr,Pot S KVAr,Pot T KVAr,Pot R KVAr faltante,Pot S KVAr faltante,Pot T KVAr faltante
Data,,,,,,,,,,,,,,,,,,,,,
2019-07-07 05:05:10,1.57,1.45,1.55,0.16,0.03,0.32,221.25,218.48,218.70,0.601649,...,0.587139,0.096264,0.016461,0.187885,0.593898,0.548460,0.556266,0.358101,0.333412,0.326155
2019-07-07 05:05:20,1.57,1.45,1.55,0.16,0.03,0.32,221.17,218.48,218.64,0.601432,...,0.586978,0.096229,0.016461,0.187833,0.593684,0.548460,0.556113,0.357971,0.333412,0.326066
2019-07-07 05:05:30,1.55,1.45,1.55,0.16,0.03,0.32,221.25,218.53,218.67,0.593985,...,0.587059,0.095038,0.016465,0.187859,0.586333,0.548585,0.556190,0.353539,0.333488,0.326111
2019-07-07 05:05:40,1.55,1.45,1.55,0.16,0.03,0.32,221.25,218.59,218.78,0.593985,...,0.587354,0.095038,0.016469,0.187953,0.586333,0.548736,0.556469,0.353539,0.333579,0.326275
2019-07-07 05:05:50,1.57,1.45,1.57,0.16,0.03,0.32,221.23,218.53,218.75,0.601595,...,0.594851,0.096255,0.016465,0.190352,0.593845,0.548585,0.563572,0.358069,0.333488,0.330439


In [13]:
Valores_ComparacaoR = fpfases[(fpfases['FP R'] > fpfases['FP T'] ) & (fpfases['FP R'] > fpfases['FP T'])]
Valores_ComparacaoS = fpfases[(fpfases['FP S'] > fpfases['FP T'] ) & (fpfases['FP S'] > fpfases['FP T'])]
Valores_ComparacaoT = fpfases[(fpfases['FP T'] > fpfases['FP R'] ) & (fpfases['FP T'] > fpfases['FP S'])]

In [14]:
Universal_p.round(5).to_csv('Universal Leituras.csv')
Universal.round(5).to_csv('Universal.csv')
fpfases.round(5).to_csv('Universal FP FORAS.csv')